<a href="https://colab.research.google.com/github/Ignacio-Ibarra/NLP-Disasters/blob/intento_1_nacho/intento_1/Test_Set_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we're gonna generate same features that we had made with the train set. 

# Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
from wordcloud import WordCloud
import PIL
import itertools
from collections import defaultdict
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import string
import re
from PIL import Image    # to import the image
from google.colab import files
from nltk.util import ngrams # function for making ngrams
import collections
import plotly.express as px

sns.set_context('paper',font_scale=1.5)




%matplotlib inline

#para suprimir notacion cientifica en los outputs
pd.options.display.float_format='{:20,.1f}'.format

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


# Reading csv file from GitHub and creating DataFrame

In [26]:
#Reading test set

test = pd.read_csv('https://raw.githubusercontent.com/Ignacio-Ibarra/NLP-Disasters/master/csv/test.csv', sep=',')
test = test[0:]
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


# Filling nulls

In [42]:
test['keyword']=test['keyword'].fillna('no_keyword')
test['location']=test['location'].fillna('no_location')

# Feature Engineering to Test Set

In [34]:
def cleanText(inputString):
  text = re.sub(r'http\S+', '', inputString)  
  text2 = re.sub(r'@\S+', '', text)
  text3 = re.sub(r'&\S+', '', text2)
  text4 = re.sub(r'Ã\S+', '', text3)
  text5 = re.sub(r'www\S+','',text4)
  text5 = re.sub(r'www\S+','',text4)
  text6 = re.sub(r'\x89Û\S+','',text4)

  return (text6)
 
test['text_clean'] = test['text'].apply(lambda x: cleanText(x))

In [35]:
def decontracted(inputString):

    phrase = re.sub(r"won't", "will not", inputString)
    phrase = re.sub(r"couldn't", "could not", inputString)
    phrase = re.sub(r"can't", "can not", inputString)
    phrase = re.sub(r"don't", "do not", inputString)
    phrase = re.sub(r"doesn't", "does not", inputString)
    phrase = re.sub(r"isn't", "is not", inputString)
    phrase = re.sub(r"mustn't", "must not", inputString)
    phrase = re.sub(r"shouldn't", "was not", inputString)
    phrase = re.sub(r"wasn't", "was not", inputString)
    phrase = re.sub(r"won't", "will not", inputString)
    
    # general
    phrase = re.sub(r" 's", " \"s", phrase)
    phrase = re.sub(r" 't", " \"t", phrase)
    phrase = re.sub(r" 'd", " \"d", phrase)
    phrase = re.sub(r" 're", " \"re", phrase)
    phrase = re.sub(r" 'll", " \"ll", phrase)
    phrase = re.sub(r" 've", " \"ve", phrase)
    phrase = re.sub(r" 'm", " \"m", phrase)

    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

test['text_clean'] = test['text_clean'].apply(lambda x: decontracted(x))

In [36]:
def deletePunctuation(inputString):
  aux=string.punctuation
  special_characters='ÃŒ©‰¥¼ªû'
  #puncts=aux.translate(str.maketrans("#@:/.","     "))
  auxstring=" " * len(aux)
  auxstring2=" " * len(special_characters)
  outputString=inputString.translate(str.maketrans(aux,auxstring))
  outputString=outputString.translate(str.maketrans(special_characters,auxstring2))
  return outputString

test['text_clean'] = test['text_clean'].apply(lambda x:deletePunctuation(x))

In [38]:
def deleteStopwords(inputString):
  inputString= inputString.lower()
  text_tokens = word_tokenize(inputString)
  tokens_without_sw = [word for word in text_tokens if not word in stopwords.words('english')]
  filtered_sentence = (" ").join(tokens_without_sw)
  return filtered_sentence

test['text_clean_no_sw'] = test['text_clean'].apply(lambda x:deleteStopwords(x))

### Tweets text features

In [40]:
def url_count(a):
  a=a.split()
  i=0
  count=0
  while i<len(a):
    if 'http:' in a[i] or 'www.' in a[i]:
      count=count+1
    i=i+1
  return count 

def mention_count(a):
  a=a.split()
  i=0
  count=0
  while i<len(a):
    
    if '@' in a[i]:
      count=count+1
    i=i+1
  return count 

def hashtag_count(a):
  a=a.split()
  i=0
  count=0
  while i<len(a):
    
    if '#' in a[i]:
      count=count+1
    i=i+1
  return count  

def numbers_count(inputString):
  count=0
  i=0
  inputString=inputString.split()
  while i < len(inputString):
    if inputString[i].isdigit() == True:
      count=count+1
    i=i+1
  return count

test['url_count'] = test['text'].apply(lambda x: url_count(x))
test['hashtag_count'] = test['text'].apply(lambda x: hashtag_count(x))
test['mention_count'] = test['text'].apply(lambda x: mention_count(x))
test['digits_count'] = test['text_clean'].apply(lambda x: numbers_count(x))
test['characters_count'] = test['text'].apply(lambda x: len(x))
test['characters_count_clean'] = test['text_clean'].apply(lambda x: len(x))
test['characters_count_clean_sw'] = test['text_clean_no_sw'].apply(lambda x: len(x))
test['word_count'] = test['text'].apply(lambda x: len(x.split()))
test['word_count_clean'] = test['text_clean'].apply(lambda x: len(x.split()))
test['word_count_clean_sw'] = test['text_clean_no_sw'].apply(lambda x: len(x.split()))
test['avg_word_len']= test['characters_count']/ test['word_count']
test['avg_word_len_clean_sw']= test['characters_count_clean_sw']/ test['word_count_clean_sw']

In [44]:
#Obtenemos el token de cada texto. 
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
test['tokenized_text']=test['text_clean'].apply(lambda x: x.lower()).apply(tt.tokenize) 

#Para cada token obtenemos el tag (clase de palabra)
test['pos_tagged_text'] = test['tokenized_text'].apply(nltk.Text).apply(nltk.pos_tag)

#En cada fila contamos la frecuencia de las clases de palabras
from collections import Counter
test['pos_tagg_counts']=test['pos_tagged_text'].apply(lambda x: Counter(tag for word,tag in x))

#Creamos un dataframe con la misma cantidad de filas que el dataframe con el que venimos trabajando. 
#Las columnas serán todas las clases de palabras existentes en la librería nltk que estén dentro de nuestras frecuencias. 
data = test['pos_tagg_counts']
word_classes_df = pd.DataFrame.from_records(data, columns = data.sum().keys())

#Dicho dataframe posee por fila el dato de la frecuencia para cada clase. 
word_classes_df.fillna(0,inplace=True)

#Obtenemos el nuevo dataframe
featured_test=pd.concat([test,word_classes_df],axis=1)

In [47]:
featured_test.to_csv('featured_test.csv', sep=',')